## Домашнее творческое задание по дисциплине "Технологии обработки больших данных" (часть 2)

Выполнил: студент группы Мамаджанов Азиз 

Вариант: __68__

    

__При решении данных задач не подразумевается использования циклов или генераторов Python в ходе работы с пакетами `numpy`, `pandas` и `dask`, если в задании не сказано обратного. Решения задач, в которых для обработки массивов `numpy`, структур `pandas` или структур `dask` используются явные циклы (без согласования с преподавателем), могут быть признаны некорректными и не засчитаны.__

__В задачах, связанных с пакетом `dask`, явно указано, какой коллекцией вы должны пользоваться при решении задачи. Все операции вычислений должны проводиться  над этой коллекцией и средствами пакета `dask`, если в задании не сказано обратного.__ 



In [1]:
import numpy as np
import sqlite3
import pandas as pd
import multiprocessing
import h5py
import dask.array as da
import dask.bag as db
import json
import dask.dataframe as dd
from collections import Counter
from numba import njit

__Задание 1__. В файле `scores.npy` содержится массив оценок рейтингов вопросов. Напишите функцию, которая считает сумму квадратов цифр числа. Векторизуйте эту функцию при помощи `numba.vectorize` и посчитайте количество чисел из файла, сумма квадратов цифр которых меньше 50.

In [2]:
scores = np.load('data/scores.npy')
scores


array([21, 27, 40, ...,  0,  2,  0], dtype=int64)

In [3]:
# Напишите функцию, которая считает сумму квадратов цифр числа
def sum_of_squares_of_digits(num):
    summ = 0
    num = abs(num)
    while num > 0:
        summ += (num % 10) ** 2
        num //= 10
    return summ



In [4]:
%time len(list(filter(lambda x: x < 50, map(sum_of_squares_of_digits, scores))))

CPU times: total: 1.08 s
Wall time: 1.1 s


594575

In [5]:
# Векторизуйте эту функцию при помощи numba.vectorize 
@njit
def sum_of_squares_of_digits(num):
    summ = 0
    num = abs(num)
    while num > 0:
        summ += (num % 10) ** 2
        num //= 10
    return summ



In [6]:
# посчитайте количество чисел из файла, сумма квадратов цифр которых меньше 50.
%time len(list(filter(lambda x: x < 50, map(sum_of_squares_of_digits, scores))))


CPU times: total: 828 ms
Wall time: 856 ms


594575

__Задание 2__. В файле `questions.csv` содержатся вопросы, заданные на сайте stackoverflow. Создайте базу данных SQLite с таблицей `Questions`. Работая с объектом-курсором, выполните запрос на языке SQL, который возращает вопросы, касающиеся фреймворка Django.

In [7]:
# Создайте базу данных SQLite с таблицей Questions.
con = sqlite3.connect("ex_2.sqlite")
cur = con.cursor()

sql = '''
DROP TABLE IF EXISTS Questions;
CREATE TABLE Questions(
    id INT,
    owner_user_id INT,
    creation_date VARCHAR,
    score INT,
    title TEXT,
    body TEXT
);
'''

cur.executescript(sql)
con.commit()


In [8]:
df = pd.read_csv('data/questions.csv')
sql = '''
INSERT INTO Questions (id, owner_user_id, creation_date, score, title, body) VALUES (?, ?, ?, ?, ?, ?)
'''
rows = [tuple(x) for x in df.to_numpy()]

cur.executemany(sql, rows)
con.commit()


In [9]:
# Работая с объектом-курсором, выполните запрос на языке SQL, который возращает вопросы, касающиеся фреймворка Django.
cur.execute("SELECT * FROM Questions WHERE Title LIKE '%Django%' OR Body LIKE '%Django%'").fetchall()


[(742,
  189,
  '2008-08-03T15:55:28Z',
  30,
  'Class views in Django',
  '<p><a href="http://www.djangoproject.com/">Django</a> view points to a function, which can be a problem if you want to change only a bit of functionality. Yes, I could have million keyword arguments and even more if statements in the function, but I was thinking more of an object oriented approach.</p>\r\\n\r\\n<p>For example, I have a page that displays a user. This page is very similar to page that displays a group, but it\'s still not so similar to just use another data model. Group also has members etc...</p>\r\\n\r\\n<p>One way would be to point views to class methods and then extend that class. Has anyone tried this approach or has any other idea? </p>'),
 (766,
  1384652,
  '2008-08-03T17:44:07Z',
  20,
  'Python and MySQL',
  '<p>I can get Python to work with Postgresql but I cannot get it to work with MySQL. The main problem is that on the shared hosting account I have I do not have the ability to inst

__Задание 3__. В файле `questions.csv` содержатся вопросы, заданные на сайте stackoverflow. Используя `multiprocessing.Pool` выясните, сколько вопросов задали в 2011 году. При решении задачи разделите файл на части, для обработки каждой из частей используйте параллельные вычисления.

In [10]:
mkdir questions


In [11]:
# При решении задачи разделите файл на части, для обработки каждой из частей используйте параллельные вычисления.
source_path = "data/questions.csv"
for i,chunk in enumerate(pd.read_csv(source_path, chunksize=50000)):
    chunk.to_csv('questions/q_{}.csv'.format(i), index=False)


In [12]:
files = []
for i in range(0, 13):
    files.append(f"questions/q_{i}.csv")
files


['questions/q_0.csv',
 'questions/q_1.csv',
 'questions/q_2.csv',
 'questions/q_3.csv',
 'questions/q_4.csv',
 'questions/q_5.csv',
 'questions/q_6.csv',
 'questions/q_7.csv',
 'questions/q_8.csv',
 'questions/q_9.csv',
 'questions/q_10.csv',
 'questions/q_11.csv',
 'questions/q_12.csv']

In [13]:
%%file year_2011.py
import pandas as pd

def year_2011(file):
    df = pd.read_csv(file, header=None)
    return len(df[df[2].str[:4]=='2011'])


Writing year_2011.py


In [14]:
from year_2011 import year_2011


In [15]:
# Используя multiprocessing.Pool выясните, сколько вопросов задали в 2011 году
with multiprocessing.Pool(processes = 13) as pool:
    result = pool.map(year_2011, files)


In [16]:
result


[9308, 31602, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [17]:
sum(result)


40910

__Задание 4__. В файле `qembeddings.hdf` находятся векторные представления заголовков вопросов, заданные на сайте stackoverflow. В файле `qtitles.pickle` находится список заголовков этих вопросов. Напишите функцию, которая для вопроса с индексом `i` возвращает количество вопросов, находящихся от него на расстоянии не более `k` (`k` - параметр функции). Выберите метрику, наиболее подходящую (на ваш взгляд) для решения этой задачи. Для работы с эмбеддингами используйте `dask.array`.

In [18]:
hdf = h5py.File("data/qembeddinbgs.h5", "r")
dset = hdf["embeddings"]
dset


<HDF5 dataset "embeddings": shape (600000, 312), type "<f4">

In [19]:
arr = da.from_array(dset)
arr.compute()


array([[-0.0559735 , -0.02404885,  0.02926788, ...,  0.02400476,
         0.03378522, -0.1039519 ],
       [ 0.00267194, -0.05379557,  0.01873641, ...,  0.02453617,
         0.0643132 , -0.11858475],
       [-0.04478475,  0.01515992,  0.00732702, ..., -0.0178924 ,
         0.09144077, -0.05961959],
       ...,
       [-0.01839547, -0.03483053,  0.10025411, ..., -0.02162502,
         0.01798907, -0.05586268],
       [-0.0631756 , -0.03707054,  0.02739069, ...,  0.01249216,
         0.08204011, -0.07276718],
       [ 0.04949792, -0.08233714, -0.00025805, ..., -0.08520836,
         0.02428205, -0.08063217]], dtype=float32)

Буду использовать Евклидово расстояние
$$r(\textbf{x},\textbf{y})=\sum_{k=1}^{n}{sqrt((x_k - y_k)^2)}, \textbf{x}, \textbf{y} \in \mathbb{R}^n$$

In [20]:
def r1(i, k):
    r1 = da.sqrt((arr - arr[:, i].reshape(-1, 1)) ** 2).sum(axis=0)
    return len(r1[r1 <= k].compute()) - 1



In [21]:
r1(0, 21000)


18

__Задание 5__. В файлах `questions_with_answers_*.jsonl` содержатся вопросы, заданные на сайте stackoverflow, и ответы на них. Разбейте вопросы на 3 группы: вопросы, на которые дали меньше 5 ответов; вопросы, на которые дали больше 5 ответов, но меньше 15 ответов; и вопросы, на которые дали 15 ответов или более. Посчитайте количество вопросов в каждой из групп. Для обработки данных используйте `dask bag`.

In [22]:
b = db.read_text('data/questions_with_answers/questions_with_answers_*.jsonl')
b.take(5)


('{"Id": 25486492, "OwnerUserId": 2384800.0, "CreationDate": "2014-08-25T12:57:02Z", "Score": 0, "Title": "Using certain rows as columns to create DataFrame", "Body": "<p>I have a data frame <em>DF</em> looking like</p>\\n\\n<pre><code>DF =\\n\\n     Date            Counterparty    Amount\\n0    2014-08-25      A               100\\n1    2014-08-25      A               200   \\n2    2014-08-25      B               300   \\n3    2014-08-26      C               400 \\n4    2014-08-27      C               500  \\n5    2014-08-27      A               600\\n6    2014-08-29      D               1000\\n</code></pre>\\n\\n<p>I want to somehow create a new data frame <em>DF_new</em>, with (unique) columns as of the rows of <em>DF</em>:s column <em>\'Counterparty\'</em> with aggregated amounts from column <em>\'Amount\'</em>. In code, a result looking like</p>\\n\\n<pre><code>DF_new =\\n\\n     Date            A            B            C            D\\n0    2014-08-25      300          300      

In [23]:
def loading_like_json_loads(x):
    try:
        return json.loads(x)
    except json.JSONDecodeError:
        return {'answers': []}



In [24]:
b = b.map(loading_like_json_loads)
b.take(5)


({'Id': 25486492,
  'OwnerUserId': 2384800.0,
  'CreationDate': '2014-08-25T12:57:02Z',
  'Score': 0,
  'Title': 'Using certain rows as columns to create DataFrame',
  'Body': "<p>I have a data frame <em>DF</em> looking like</p>\n\n<pre><code>DF =\n\n     Date            Counterparty    Amount\n0    2014-08-25      A               100\n1    2014-08-25      A               200   \n2    2014-08-25      B               300   \n3    2014-08-26      C               400 \n4    2014-08-27      C               500  \n5    2014-08-27      A               600\n6    2014-08-29      D               1000\n</code></pre>\n\n<p>I want to somehow create a new data frame <em>DF_new</em>, with (unique) columns as of the rows of <em>DF</em>:s column <em>'Counterparty'</em> with aggregated amounts from column <em>'Amount'</em>. In code, a result looking like</p>\n\n<pre><code>DF_new =\n\n     Date            A            B            C            D\n0    2014-08-25      300          300          0         

In [25]:
'''
Разбейте вопросы на 3 группы: 
вопросы, на которые дали меньше 5 ответов; 
вопросы, на которые дали больше 5 ответов, но меньше 15 ответов; 
и вопросы, на которые дали 15 ответов или более
'''
def groups(bag):
        if len(bag['answers']) < 5 and len(bag['answers']) != 0:
            return '1st group'
        elif len(bag['answers']) >= 5 and len(bag['answers']) < 15:
            return '2nd group'
        else:
            return '3st group'



In [26]:
# Посчитайте количество вопросов в каждой из групп.
Counter(b.map(groups).compute())


Counter({'1st group': 340954, '3st group': 45056, '2nd group': 13997})

__Задание 6__. В файле `questions.csv` содержатся вопросы, заданные на сайте stackoverflow.Проанализируйте и сделайте вывод: существует ли корреляция между длиной описания вопроса и оценкой score этого вопроса. Для обработки данных используйте `dask.dataframe`.

In [2]:
df = dd.read_csv('data/questions.csv')
df.head()


,Id,OwnerUserId,CreationDate,Score,Title,Body
0,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...
1,502,147.0,2008-08-02T17:01:58Z,27,Get a preview JPEG of a PDF on Windows?,<p>I have a cross-platform (Python) applicatio...
2,535,154.0,2008-08-02T18:43:54Z,40,Continuous Integration System for a Python Cod...,<p>I'm starting work on a hobby project with a...
3,594,116.0,2008-08-03T01:15:08Z,25,cx_Oracle: How do I iterate over a result set?,<p>There are several ways to iterate over a re...
4,683,199.0,2008-08-03T13:19:16Z,28,Using 'in' to match an attribute of Python obj...,<p>I don't remember whether I was dreaming or ...


In [3]:
df['Body_2'] = df['Body'].str.len()
df.head()


,Id,OwnerUserId,CreationDate,Score,Title,Body,Body_2
0,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...,456
1,502,147.0,2008-08-02T17:01:58Z,27,Get a preview JPEG of a PDF on Windows?,<p>I have a cross-platform (Python) applicatio...,317
2,535,154.0,2008-08-02T18:43:54Z,40,Continuous Integration System for a Python Cod...,<p>I'm starting work on a hobby project with a...,627
3,594,116.0,2008-08-03T01:15:08Z,25,cx_Oracle: How do I iterate over a result set?,<p>There are several ways to iterate over a re...,92
4,683,199.0,2008-08-03T13:19:16Z,28,Using 'in' to match an attribute of Python obj...,<p>I don't remember whether I was dreaming or ...,321


In [4]:
# Проанализируйте и сделайте вывод: существует ли корреляция между длиной описания вопроса и оценкой score этого вопроса.
df[['Score', 'Body_2']].corr().compute()


,Score,Body_2
Score,1.000000,-0.022159
Body_2,-0.022159,1.000000


Корреляции нет, тк коэффициент близок к нулю